In [1]:
import torch
import argparse
from model import MODEL
from run import train, test
import numpy as np
import torch.optim as optim
from dataloader import getDataLoader
from earlystop import EarlyStopping

In [2]:
gpu = 0
patience = 5
max_iter = 100  # number of iterations
init_lr = 0.005  # initial learning rate
lr_decay = 0.75  # learning rate decay
final_lr = 1E-5  # learning rate will not decrease after hitting this threshold
dropout = 0.3
exercise_embed_dim = 64  # hidden state dim for LSTM
hidden_dim = 64
layer_num = 2  # layer number for LSTM
max_step = 100  # the allowed maximum length of a sequence
maxgradnorm = 50.0  # maximum gradient norm
Lamda = 1  # hyper-parameter Lamda
alpha = 0.2  # alpha
num_heads = 4  # number of head attentions
mode = 3  # mode of integration function
# 1: ca / 2: mul / 3: ca mul / 4: rasch
fold = '1'  # number of fold
dataset = 'assist2009_B'

#when dataset is assist2009_B
batch_size = 32
n_knowledge_concept = 110  # the kinds of unique questions in the dataset
n_exercise = 16891  # the number of unique questions in the dataset
data_dir = './data/assist2009_B'  # data directory
data_name = 'assist2009_B'  # data set name

memory_size = n_knowledge_concept
lr = init_lr
memory_key_state_dim = exercise_embed_dim
memory_value_state_dim = exercise_embed_dim * 2

train_data_path = data_dir + "/" + data_name + "_train" + fold + ".csv"
valid_data_path = data_dir + "/" + data_name + "_valid" + fold + ".csv"
test_data_path = data_dir + "/" + data_name + "_test" + fold + ".csv"

In [3]:
import numpy as np
import itertools
class DataReader():
    def __init__(self, train_path, valid_path, test_path, maxstep, numofques):
        self.train_path = train_path
        self.valid_path = valid_path
        self.test_path = test_path
        self.maxstep = maxstep
        self.numofques = numofques

    def getTrainData(self):
        print('loading train data...')
        q_data= []
        qa_data =[]
        p_data= []
        trainData = []
        batch = 0
        with open(self.train_path, 'r') as train:
            for student, problem, ques, ans in itertools.zip_longest(*[train] * 4):
                batch = batch +1
                try:
                    problem = [int(p) for p in problem.strip().strip(',').split(',')]
                    ques = [int(q) for q in ques.strip().strip(',').split(',')]
                    ans = [int(a) for a in ans.strip().strip(',').split(',')]
                except:
                    print(batch)
                length = np.size(ques)
                slices = length//self.maxstep + (1 if length%self.maxstep > 0 else 0)
                for i in range(slices):
                    q_temp = np.zeros(shape=[self.maxstep])
                    qa_temp = np.zeros(shape=[self.maxstep])
                    p_temp = np.zeros(shape=[self.maxstep])
                    if length > 0:
                        if length >= self.maxstep:
                            l = self.maxstep
                        else:
                            l = length
                        for j in range(l):
                            q_temp[j] = ques[i*self.maxstep + j]
                            qa_temp[j] = ans[i*self.maxstep + j]
                            p_temp[j] = problem[i*self.maxstep + j]
                        length = length - self.maxstep
                    q_data.append(q_temp.tolist())
                    qa_data.append(qa_temp.tolist())
                    p_data.append(p_temp.tolist())
            print('train--question done: ' + str(np.array(q_data).shape))
            print('train--question_ans done: ' + str(np.array(qa_data).shape))
            print('train--problem done: ' + str(np.array(p_data).shape))
            return np.array(q_data).astype(float), np.array(qa_data).astype(float), np.array(p_data).astype(float)

    def getValidData(self):
        print('loading valid data...')
        q_data= []
        qa_data =[]
        p_data= []
        validData = []
        batch = 0
        with open(self.valid_path, 'r') as valid:
            for student, problem, ques, ans in itertools.zip_longest(*[valid] * 4):
                batch = batch +1
                try:
                    problem = [int(p) for p in problem.strip().strip(',').split(',')]
                    ques = [int(q) for q in ques.strip().strip(',').split(',')]
                    ans = [int(a) for a in ans.strip().strip(',').split(',')]
                except:
                    print(batch)
                length = np.size(ques)
                slices = length//self.maxstep + (1 if length%self.maxstep > 0 else 0)
                for i in range(slices):
                    q_temp = np.zeros(shape=[self.maxstep])
                    qa_temp = np.zeros(shape=[self.maxstep])
                    p_temp = np.zeros(shape=[self.maxstep])
                    if length > 0:
                        if length >= self.maxstep:
                            l = self.maxstep
                        else:
                            l = length
                        for j in range(l):
                            q_temp[j] = ques[i*self.maxstep + j]
                            qa_temp[j] = ans[i*self.maxstep + j]
                            p_temp[j] = problem[i*self.maxstep + j]
                        length = length - self.maxstep
                    q_data.append(q_temp.tolist())
                    qa_data.append(qa_temp.tolist())
                    p_data.append(p_temp.tolist())
            print('valid--question done: ' + str(np.array(q_data).shape))
            print('valid--question_ans done: ' + str(np.array(qa_data).shape))
            print('valid--problem done: ' + str(np.array(p_data).shape))
            return np.array(q_data).astype(float), np.array(qa_data).astype(float), np.array(p_data).astype(float)


    def getTestData(self):
        print('loading test data...')
        testData = []
        q_data = []
        qa_data = []
        p_data = []
        zero = [0 for i in range(self.numofques * 2)]
        batch = 0
        with open(self.test_path, 'r') as test:
            for student, problem, ques, ans in itertools.zip_longest(*[test] * 4):
                #length = int(length.strip().strip(','))
                batch = batch + 1
                try:
                    problem = [int(p) for p in problem.strip().strip(',').split(',')]
                    ques = [int(q) for q in ques.strip().strip(',').split(',')]
                    ans = [int(a) for a in ans.strip().strip(',').split(',')]
                except:
                    print(batch)
                length = np.size(ques)
                slices = length // self.maxstep + (1 if length % self.maxstep > 0 else 0)
                for i in range(slices):
                    q_temp = np.zeros(shape=[self.maxstep])
                    qa_temp = np.zeros(shape=[self.maxstep])
                    p_temp = np.zeros(shape=[self.maxstep])
                    if length > 0:
                        if length >= self.maxstep:
                            l = self.maxstep
                        else:
                            l = length
                        for j in range(l):
                            q_temp[j] = ques[i * self.maxstep + j]
                            qa_temp[j] = ans[i * self.maxstep + j]
                            p_temp[j] = problem[i * self.maxstep + j]
                        length = length - self.maxstep
                    q_data.append(q_temp.tolist())
                    qa_data.append(qa_temp.tolist())
                    p_data.append(p_temp.tolist())
            print('test--question done: ' + str(np.array(q_data).shape))
            print('test--question_ans done: ' + str(np.array(qa_data).shape))
            print('test--problem done: ' + str(np.array(p_data).shape))
            return np.array(q_data).astype(float), np.array(qa_data).astype(float), np.array(p_data).astype(float)



In [4]:
import sys
sys.path.append('/')
from readdata import DataReader
import numpy as np
from tqdm import tqdm


def getDataLoader(train_data_path, valid_data_path, test_data_path, max_step, n_knowledge_concept):
    handle = DataReader(train_data_path, valid_data_path, test_data_path, max_step, n_knowledge_concept)

    kc_data_train, respose_data_train, exercise_data_train = handle.getTrainData()
    kc_data_valid, respose_data_valid, exercise_data_valid = handle.getValidData()
    kc_data_test, respose_data_test, exercise_data_test = handle.getTestData()

    return kc_data_train, respose_data_train, exercise_data_train,kc_data_valid, respose_data_valid, exercise_data_valid, kc_data_test, respose_data_test, exercise_data_test



In [5]:
train_kc_data, train_respond_data, train_exercise_data, \
        valid_kc_data, valid_respose_data, valid_exercise_data, \
        test_kc_data, test_respose_data, test_exercise_data \
        = getDataLoader(train_data_path, valid_data_path, test_data_path, max_step, n_knowledge_concept)

loading train data...
train--question done: (3293, 100)
train--question_ans done: (3293, 100)
train--problem done: (3293, 100)
loading valid data...
valid--question done: (1038, 100)
valid--question_ans done: (1038, 100)
valid--problem done: (1038, 100)
loading test data...
test--question done: (1032, 100)
test--question_ans done: (1032, 100)
test--problem done: (1032, 100)


In [128]:
train_exercise_respond_data = train_respond_data * \
    n_exercise + train_exercise_data
valid_exercise_respose_data = valid_respose_data * \
    n_exercise + valid_exercise_data
test_exercise_respose_data = test_respose_data * \
    n_exercise + test_exercise_data
adj_exercise_kc = np.loadtxt(data_dir + "/exercise_kc_map.txt")
adj_exercise_kc = utils.varible(torch.from_numpy(adj_exercise_kc), gpu)


In [7]:
len(np.random.permutation(train_exercise_data.shape[0]))


3293

In [8]:
import torch
import torch.nn as nn
import numpy as np
import utils as utils
import embedd_loss as embedd_loss
from torch.autograd import Variable
import torch.nn.functional as F

In [9]:
n_exercise = n_exercise
n_kc = n_knowledge_concept
batch_size = batch_size
exercise_embed_dim = exercise_embed_dim
student_num = None
nheads = num_heads
alpha = alpha
dropout = dropout
# params = params
mode = mode
Lamda = Lamda
hidden_dim = hidden_dim
layer_num = layer_num

In [10]:
class Exercise_KC_GraphAttentionLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout, alpha, mode, concat=True):
        super(Exercise_KC_GraphAttentionLayer, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat
        self.mode =mode
        self.W1 = nn.Parameter(torch.empty(size=(in_features, out_features)))
        nn.init.kaiming_normal(self.W1)

        if mode == 1 or mode == 3:
            self.reduceDim = nn.Linear(in_features * 2, self.out_features, bias=True)
            nn.init.kaiming_normal(self.reduceDim.weight)
            nn.init.constant(self.reduceDim.bias, 0)
        if mode != 1:
            self.E = nn.Parameter(torch.empty(size=(in_features, out_features)))
            nn.init.kaiming_normal(self.E)
        if mode ==4:
            self.U = nn.Parameter(torch.empty(size=(in_features, 1)))
            nn.init.kaiming_normal(self.U)

        self.a = nn.Parameter(torch.empty(size=(2*out_features, 1)))
        nn.init.kaiming_normal(self.a)

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, exercise_h, kc_h, adj_exercise_kc):
        if self.concat:
            kc_Wh = torch.mm(kc_h, self.W1)
            exercise_Wh = torch.mm(exercise_h, self.W1)
            a_input = self._prepare_attentional_mechanism_input(kc_Wh, exercise_Wh)
            e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))
            zero_vec = -9e15 * torch.ones_like(e)
            attention = torch.where(adj_exercise_kc > 0, e, zero_vec)
            attention = F.softmax(attention, dim=1)
            new_kc_embed = torch.matmul(attention, kc_Wh)

            if self.mode == 1:
                exercises_embedd = torch.cat((new_kc_embed, exercise_h), dim=1)
                exercises_embedd = self.reduceDim(exercises_embedd)
            if self.mode == 2:
                exercise_Eh = torch.mm(exercise_h, self.E)
                exercises_embedd = new_kc_embed.mul(exercise_Eh)
            if self.mode == 3:
                exercise_Eh = torch.mm(exercise_h, self.E)
                exercises_embedd = torch.cat((new_kc_embed, new_kc_embed.mul(exercise_Eh)), dim=1)
                exercises_embedd = self.reduceDim(exercises_embedd)
            if self.mode == 4:
                u = torch.mm(exercise_h, self.U)
                d_kt = torch.mm(new_kc_embed, self.E)
                exercises_embedd = new_kc_embed + u * d_kt
            return F.elu(exercises_embedd)

    def _prepare_attentional_mechanism_input(self, kc_Wh, exercise_Wh):
        N_kc = kc_Wh.size()[0]
        N_exercise = exercise_Wh.size()[0]
        Wh_repeated_in_chunks = exercise_Wh.repeat_interleave(N_kc, dim=0)
        Wh_repeated_alternating = kc_Wh.repeat(N_exercise, 1)
        all_combinations_matrix = torch.cat([Wh_repeated_in_chunks, Wh_repeated_alternating], dim=1)
        return all_combinations_matrix.view(N_exercise, N_kc, 2 * self.out_features)

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'

In [130]:
lstm = nn.LSTM(exercise_embed_dim * 2, hidden_dim,
               layer_num, batch_first=True, dropout=dropout)
fc1 = nn.Linear(exercise_embed_dim + hidden_dim, hidden_dim, bias=True)
fc2 = nn.Linear(hidden_dim, 1, bias=True)
rel = nn.ReLU()

exercise_embed = nn.Parameter(torch.randn(n_exercise, exercise_embed_dim))
kc_embed = nn.Parameter(torch.randn(n_kc, exercise_embed_dim))



In [129]:
dropout = dropout
in_features = exercise_embed_dim
out_features = exercise_embed_dim
alpha = alpha
concat = True
mode = mode
W1 = nn.Parameter(torch.empty(size=(in_features, out_features)))
nn.init.kaiming_normal(W1)

/tmp/ipykernel_650/995484167.py:8: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(W1)


Parameter containing:
tensor([[ 0.1202,  0.0214, -0.0699,  ...,  0.0381, -0.3796,  0.1299],
        [-0.2248, -0.1405,  0.1015,  ..., -0.1020,  0.0535, -0.0760],
        [-0.2335,  0.0681, -0.0628,  ..., -0.1022,  0.0834, -0.4411],
        ...,
        [-0.0351, -0.3010,  0.0809,  ..., -0.2111, -0.2437,  0.0782],
        [ 0.3125, -0.3680, -0.0881,  ...,  0.0310, -0.0940,  0.3209],
        [-0.2887, -0.1861, -0.1339,  ...,  0.0710, -0.0222,  0.0392]],
       requires_grad=True)

In [131]:
reduceDim = nn.Linear(in_features * 2, out_features, bias=True)
nn.init.kaiming_normal(reduceDim.weight)
nn.init.constant(reduceDim.bias, 0)

E = nn.Parameter(torch.empty(size=(in_features, out_features)))
nn.init.kaiming_normal(E)

U = nn.Parameter(torch.empty(size=(in_features, 1)))
nn.init.kaiming_normal(U)

a = nn.Parameter(torch.empty(size=(2*out_features, 1)))
nn.init.kaiming_normal(a)

leakyrelu = nn.LeakyReLU(alpha)

/tmp/ipykernel_650/1761267380.py:2: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(reduceDim.weight)
/tmp/ipykernel_650/1761267380.py:3: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(reduceDim.bias, 0)
/tmp/ipykernel_650/1761267380.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(E)
/tmp/ipykernel_650/1761267380.py:9: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(U)
/tmp/ipykernel_650/1761267380.py:12: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(a)


In [14]:
exercise_embed.shape

torch.Size([16891, 64])

In [15]:
def _prepare_attentional_mechanism_input(kc_Wh, exercise_Wh):
    N_kc = kc_Wh.size()[0]
    N_exercise = exercise_Wh.size()[0]
    Wh_repeated_in_chunks = exercise_Wh.repeat_interleave(N_kc, dim=0)
    Wh_repeated_alternating = kc_Wh.repeat(N_exercise, 1)
    all_combinations_matrix = torch.cat(
        [Wh_repeated_in_chunks, Wh_repeated_alternating], dim=1)
    return all_combinations_matrix.view(N_exercise, N_kc, 2 * out_features)


In [132]:
kc_one_seq = train_respond_data[0:32, :]
exercise_one_seq = train_exercise_data[0:32, :]
exercise_respond_batch_seq = train_exercise_respond_data[0:32, :]
target = train_exercise_respond_data[0:32, :]
target = (target - 1) / n_exercise
target = np.floor(target)
input_kc = utils.varible(torch.LongTensor(kc_one_seq), gpu)
input_exercise = utils.varible(torch.LongTensor(exercise_one_seq), gpu)
input_exercise_respond = utils.varible(
    torch.LongTensor(exercise_respond_batch_seq), gpu)
target = utils.varible(torch.FloatTensor(target), gpu)
target.shape
target = target[:, 1:]
target.shape

torch.Size([32, 99])

In [68]:
a = torch.tensor(np.random.randint(5, size = (3,4)))
print(a)
b = a.repeat_interleave(3, dim=0)
print(b)
c = a.repeat_interleave(3, dim=1)
print(c)
d = a.repeat(3,1)
print(d)
e = torch.cat([a, b], dim=0)
print(e)
f = torch.cat([a, c], dim=1)
print(f)

tensor([[3, 2, 1, 0],
        [0, 4, 2, 0],
        [2, 0, 4, 0]])
tensor([[3, 2, 1, 0],
        [3, 2, 1, 0],
        [3, 2, 1, 0],
        [0, 4, 2, 0],
        [0, 4, 2, 0],
        [0, 4, 2, 0],
        [2, 0, 4, 0],
        [2, 0, 4, 0],
        [2, 0, 4, 0]])
tensor([[3, 3, 3, 2, 2, 2, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 4, 4, 4, 2, 2, 2, 0, 0, 0],
        [2, 2, 2, 0, 0, 0, 4, 4, 4, 0, 0, 0]])
tensor([[3, 2, 1, 0],
        [0, 4, 2, 0],
        [2, 0, 4, 0],
        [3, 2, 1, 0],
        [0, 4, 2, 0],
        [2, 0, 4, 0],
        [3, 2, 1, 0],
        [0, 4, 2, 0],
        [2, 0, 4, 0]])
tensor([[3, 2, 1, 0],
        [0, 4, 2, 0],
        [2, 0, 4, 0],
        [3, 2, 1, 0],
        [3, 2, 1, 0],
        [3, 2, 1, 0],
        [0, 4, 2, 0],
        [0, 4, 2, 0],
        [0, 4, 2, 0],
        [2, 0, 4, 0],
        [2, 0, 4, 0],
        [2, 0, 4, 0]])
tensor([[3, 2, 1, 0, 3, 3, 3, 2, 2, 2, 1, 1, 1, 0, 0, 0],
        [0, 4, 2, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 0, 0, 0],
        [2, 0, 

In [104]:
exercise_embed = nn.Parameter(torch.randn(n_exercise, exercise_embed_dim))
kc_embed.shape

torch.Size([110, 64])

In [146]:
kc_embed.shape
exercise_embed.shape
kc_Wh = torch.mm(kc_embed, W1)
exercise_Wh = torch.mm(exercise_embed, W1)
N_kc = kc_Wh.size()[0]
N_exercise = exercise_Wh.size()[0]
Wh_repeated_in_chunks = exercise_Wh.repeat_interleave(N_kc, dim=0)
# 110개의 knowledge_concept에 대해 문항별 weight 생성
Wh_repeated_alternating = kc_Wh.repeat(N_exercise, 1)
# 16891개의 exercise에 대해 kc별 weight 생성
all_combinations_matrix = torch.cat(
    [Wh_repeated_in_chunks, Wh_repeated_alternating], dim=1)
a_input = all_combinations_matrix.view(N_exercise, N_kc, 2*out_features)


In [149]:
print(Wh_repeated_alternating.shape)
print(Wh_repeated_in_chunks.shape)
print(a_input.shape)

torch.Size([1858010, 64])
torch.Size([1858010, 64])
torch.Size([16891, 110, 128])


In [134]:
a = nn.Parameter(torch.empty(size=(2*64, 1)))
nn.init.kaiming_normal(a)
a.shape

/tmp/ipykernel_650/2829987827.py:2: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(a)


torch.Size([128, 1])

In [144]:
print(a_input.shape)
print(a.shape)
torch.matmul(a_input, a).shape
e = leakyrelu(torch.matmul(a_input, a).squeeze(2)).cuda()
zero_vec = -9e15 * torch.ones_like(e).cuda()
print(e.shape)
print(zero_vec.shape)
adj_exercise_kc.shape
attention = torch.where(adj_exercise_kc > 0, e, zero_vec)
print(attention.shape)
print(kc_Wh.shape)

torch.Size([16891, 110, 128])
torch.Size([128, 1])
torch.Size([16891, 110])
torch.Size([16891, 110])
torch.Size([16891, 110])
torch.Size([110, 64])


In [205]:
e = leakyrelu(torch.matmul(a_input, a).squeeze(2)).cuda()
zero_vec = -9e15 * torch.ones_like(e).cuda()
attention = torch.where(adj_exercise_kc > 0, e, zero_vec)
attention = F.softmax(attention, dim=1)
new_kc_embed = torch.matmul(attention, kc_Wh.cuda())

In [152]:
print(new_kc_embed.shape)
# Exercise_KC_GraphAttentionLayer(self.exercise_embed_dim, self.exercise_embed_dim, dropout=self.dropout, alpha=self.alpha, mode=self.mode,
# concat=True) for _ in range(self.nheads)]
kc_h = kc_embed
exercise_h = exercise_embed
kc_Wh = torch.mm(kc_h, W1) #(110, 64)
exercise_Wh = torch.mm(exercise_h, W1) #(16891, 64)


torch.Size([16891, 64])


In [163]:
exercise_Eh = torch.mm(exercise_h, E)
exercises_embedd = torch.cat(
    (new_kc_embed, new_kc_embed.mul(exercise_Eh.cuda())), dim=1)
print(exercises_embedd.shape)
reduceDim = reduceDim.cuda()
exercises_embedd = reduceDim(exercises_embedd)
print(exercises_embedd.shape)


torch.Size([16891, 128])
torch.Size([16891, 64])


In [17]:
exercise_Wh.repeat_interleave(110, dim=0).shape
# 110개의 knowledge_concept에 대해 문항별 weight 생성
kc_Wh.repeat(16891, 1).shape
# 16891개의 exercise에 대해 

NameError: name 'exercise_Wh' is not defined

In [ ]:
# Exercise_KC_GraphAttentionLayer(self.exercise_embed_dim, self.exercise_embed_dim, dropout=self.dropout, alpha=self.alpha, mode=self.mode,
# concat=True) for _ in range(self.nheads)]
kc_h = kc_embed
exercise_h = exercise_embed
kc_Wh = torch.mm(kc_h, W1) #(110, 64)
exercise_Wh = torch.mm(exercise_h, W1) #(16891, 64)


In [164]:
class Exercise_KC_GraphAttentionLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout, alpha, mode, concat=True):
        super(Exercise_KC_GraphAttentionLayer, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat
        self.mode = mode
        self.W1 = nn.Parameter(torch.empty(size=(in_features, out_features)))
        nn.init.kaiming_normal(self.W1)

        if mode == 1 or mode == 3:
            self.reduceDim = nn.Linear(
                in_features * 2, self.out_features, bias=True)
            nn.init.kaiming_normal(self.reduceDim.weight)
            nn.init.constant(self.reduceDim.bias, 0)
        if mode != 1:
            self.E = nn.Parameter(torch.empty(
                size=(in_features, out_features)))
            nn.init.kaiming_normal(self.E)
        if mode == 4:
            self.U = nn.Parameter(torch.empty(size=(in_features, 1)))
            nn.init.kaiming_normal(self.U)

        self.a = nn.Parameter(torch.empty(size=(2*out_features, 1)))
        nn.init.kaiming_normal(self.a)

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, exercise_h, kc_h, adj_exercise_kc):
        if self.concat:
            kc_Wh = torch.mm(kc_h, self.W1)
            exercise_Wh = torch.mm(exercise_h, self.W1)
            a_input = self._prepare_attentional_mechanism_input(
                kc_Wh, exercise_Wh)
            e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))
            zero_vec = -9e15 * torch.ones_like(e)
            attention = torch.where(adj_exercise_kc > 0, e, zero_vec)
            attention = F.softmax(attention, dim=1)
            new_kc_embed = torch.matmul(attention, kc_Wh)

            if self.mode == 1:
                exercises_embedd = torch.cat((new_kc_embed, exercise_h), dim=1)
                exercises_embedd = self.reduceDim(exercises_embedd)
            if self.mode == 2:
                exercise_Eh = torch.mm(exercise_h, self.E)
                exercises_embedd = new_kc_embed.mul(exercise_Eh)
            if self.mode == 3:
                exercise_Eh = torch.mm(exercise_h, self.E)
                exercises_embedd = torch.cat(
                    (new_kc_embed, new_kc_embed.mul(exercise_Eh)), dim=1)
                exercises_embedd = self.reduceDim(exercises_embedd)
            if self.mode == 4:
                u = torch.mm(exercise_h, self.U)
                d_kt = torch.mm(new_kc_embed, self.E)
                exercises_embedd = new_kc_embed + u * d_kt
            return F.elu(exercises_embedd)

    def _prepare_attentional_mechanism_input(self, kc_Wh, exercise_Wh):
        N_kc = kc_Wh.size()[0]
        N_exercise = exercise_Wh.size()[0]
        Wh_repeated_in_chunks = exercise_Wh.repeat_interleave(N_kc, dim=0)
        Wh_repeated_alternating = kc_Wh.repeat(N_exercise, 1)
        all_combinations_matrix = torch.cat(
            [Wh_repeated_in_chunks, Wh_repeated_alternating], dim=1)
        return all_combinations_matrix.view(N_exercise, N_kc, 2 * self.out_features)


In [172]:
exercise_kc_attentions = [
    Exercise_KC_GraphAttentionLayer(exercise_embed_dim, exercise_embed_dim, dropout=dropout, alpha=alpha, mode=mode, concat=True) for _ in range(nheads)]
exercise_embedding = torch.cat([att(exercise_embed, kc_embed, adj_exercise_kc) for att in exercise_kc_attentions], dim=1)

/tmp/ipykernel_650/348885505.py:11: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(self.W1)
/tmp/ipykernel_650/348885505.py:16: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(self.reduceDim.weight)
/tmp/ipykernel_650/348885505.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(self.reduceDim.bias, 0)
/tmp/ipykernel_650/348885505.py:21: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(self.E)
/tmp/ipykernel_650/348885505.py:27: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(self.a)


RuntimeError: Expected condition, x and y to be on the same device, but condition is on cuda:0 and x and y are on cpu and cpu respectively

In [193]:
import torch

# Create a tensor of size (3, 3, 4)
x = torch.rand(3, 3, 4)

# Reshape the tensor to size (2, 6)


# Compute the mean of all the elements in the tensor
mean_x = x.mean(2)


print(mean_x.shape)


torch.Size([3, 3])


In [206]:
exercise_embedding_add_zero = torch.cat(
            [utils.varible(torch.zeros(1, mean_x.shape[1]), gpu), mean_x.cuda()], dim=0)
exercise_embedding_add_zero.shape

torch.Size([4, 3])

In [214]:
import torch

# Create a tensor of size (3, 3, 4)
x = torch.rand(4, 6)
chunk_x = torch.chunk(x, 2, 0)
print(x)
print(chunk_x)

tensor([[0.1481, 0.2053, 0.4924, 0.1888, 0.2748, 0.9214],
        [0.0581, 0.0780, 0.5755, 0.3228, 0.9116, 0.3702],
        [0.8120, 0.2454, 0.1920, 0.9954, 0.2558, 0.3684],
        [0.6936, 0.8881, 0.7237, 0.2804, 0.1664, 0.3215]])
(tensor([[0.1481, 0.2053, 0.4924, 0.1888, 0.2748, 0.9214],
        [0.0581, 0.0780, 0.5755, 0.3228, 0.9116, 0.3702]]), tensor([[0.8120, 0.2454, 0.1920, 0.9954, 0.2558, 0.3684],
        [0.6936, 0.8881, 0.7237, 0.2804, 0.1664, 0.3215]]))


In [257]:
seqlen = input_exercise.shape[1]
slice_exercise_data = torch.chunk(input_exercise, seqlen, 1)
print(input_exercise)
print(slice_exercise_data[0].squeeze(1).unsqueeze(1))
cat_slice_exercise_data = torch.cat([slice_exercise_data[0].squeeze(1).unsqueeze(1),slice_exercise_data[1].squeeze(1).unsqueeze(1)], 1)
cat_slice_exercise_data.size(0)

tensor([[ 1366,  1424,  1413,  ...,     0,     0,     0],
        [ 6989,  7098,  7069,  ...,  5580,  5759,  5553],
        [ 5695,  5806,  5622,  ...,  8711, 12377,  5533],
        ...,
        [ 5980,  6383,  6464,  ..., 13477, 13488, 13487],
        [13479, 13476, 13505,  ..., 14834, 14835, 14836],
        [14230, 14369, 14370,  ..., 15978, 15988, 15989]], device='cuda:0')
tensor([[ 1366],
        [ 6989],
        [ 5695],
        [ 8682],
        [11405],
        [13333],
        [ 6284],
        [11423],
        [ 8942],
        [ 7063],
        [12489],
        [12486],
        [ 7422],
        [ 8075],
        [ 5953],
        [ 7057],
        [ 5940],
        [14705],
        [ 1994],
        [  376],
        [ 4527],
        [15304],
        [12898],
        [12466],
        [13365],
        [ 3637],
        [ 5055],
        [ 2183],
        [ 3045],
        [ 5980],
        [13479],
        [14230]], device='cuda:0')


32

In [254]:
slice_exercise_data = torch.chunk(input_exercise, seqlen, 1)
len(slice_exercise_data)

100

In [286]:
x = torch.rand(32, 99, 1)
print(x.shape)
x_ = x.squeeze(-1)
print(x_.shape)
print(x)
print(x_)

torch.Size([32, 99, 1])
torch.Size([32, 99])
tensor([[[0.7773],
         [0.6655],
         [0.5498],
         ...,
         [0.8459],
         [0.3336],
         [0.6329]],

        [[0.2650],
         [0.6678],
         [0.6939],
         ...,
         [0.6394],
         [0.6408],
         [0.4079]],

        [[0.9580],
         [0.2401],
         [0.3020],
         ...,
         [0.8199],
         [0.8232],
         [0.0746]],

        ...,

        [[0.0884],
         [0.3532],
         [0.4980],
         ...,
         [0.7674],
         [0.3552],
         [0.9580]],

        [[0.9456],
         [0.3261],
         [0.1519],
         ...,
         [0.3355],
         [0.1505],
         [0.1128]],

        [[0.7458],
         [0.1261],
         [0.2113],
         ...,
         [0.2993],
         [0.9027],
         [0.7175]]])
tensor([[0.7773, 0.6655, 0.5498,  ..., 0.8459, 0.3336, 0.6329],
        [0.2650, 0.6678, 0.6939,  ..., 0.6394, 0.6408, 0.4079],
        [0.9580, 0.2401, 0.3020, 

In [293]:
target_1d = target
mask = target_1d.ge(0)
print(x_.shape)
print(mask.shape)
filtered_pred = torch.masked_select(x_, mask)
filtered_pred.shape
filtered_target = torch.masked_select(target_1d, mask)
filtered_target.shape

torch.Size([32, 99])
torch.Size([32, 99])


torch.Size([1768])

In [295]:
predict_loss = torch.nn.functional.binary_cross_entropy_with_logits(filtered_pred.cuda(), filtered_target.cuda())
predict_loss

tensor(0.6900, device='cuda:0')

In [299]:
exercise_embed.shape
kc_embed.shape

torch.Size([110, 64])